In [1]:
# testing the SQL connection
from src.database_connector import DatabaseAgent

In [2]:
username = 'root'   
password = 'core'  
hostname = 'localhost'
database_name = 'classicmodels'  

sql_agent = DatabaseAgent(host_name=hostname,port=3306,username=username,database_name=database_name,password=password, database_type="MySQL")

Connection to MySQL database successful!


In [3]:
sql_agent.grab_table_names()

['customers',
 'employees',
 'offices',
 'orderdetails',
 'orders',
 'payments',
 'productlines',
 'products']

In [4]:
sql_agent.grab_table_schema(sql_agent.grab_table_names())

[Table(name='customers', columns=[TableColumn(name='customerNumber', dtype='int'), TableColumn(name='customerName', dtype='varchar'), TableColumn(name='contactLastName', dtype='varchar'), TableColumn(name='contactFirstName', dtype='varchar'), TableColumn(name='phone', dtype='varchar'), TableColumn(name='addressLine1', dtype='varchar'), TableColumn(name='addressLine2', dtype='varchar'), TableColumn(name='city', dtype='varchar'), TableColumn(name='state', dtype='varchar'), TableColumn(name='postalCode', dtype='varchar'), TableColumn(name='country', dtype='varchar'), TableColumn(name='salesRepEmployeeNumber', dtype='int'), TableColumn(name='creditLimit', dtype='decimal')], pks=None, fks=None),
 Table(name='employees', columns=[TableColumn(name='employeeNumber', dtype='int'), TableColumn(name='lastName', dtype='varchar'), TableColumn(name='firstName', dtype='varchar'), TableColumn(name='extension', dtype='varchar'), TableColumn(name='email', dtype='varchar'), TableColumn(name='officeCode',

In [5]:
from src.promot_format import PromptFormatterV1

In [6]:
prompt_format = PromptFormatterV1(tables=sql_agent.grab_table_schema(sql_agent.grab_table_names()))

In [7]:
prompt_format(question="H")

CREATE TABLE customers (
    customerNumber int,
    customerName varchar,
    contactLastName varchar,
    contactFirstName varchar,
    phone varchar,
    addressLine1 varchar,
    addressLine2 varchar,
    city varchar,
    state varchar,
    postalCode varchar,
    country varchar,
    salesRepEmployeeNumber int,
    creditLimit decimal
)

CREATE TABLE employees (
    employeeNumber int,
    lastName varchar,
    firstName varchar,
    extension varchar,
    email varchar,
    officeCode varchar,
    reportsTo int,
    jobTitle varchar
)

CREATE TABLE offices (
    officeCode varchar,
    city varchar,
    phone varchar,
    addressLine1 varchar,
    addressLine2 varchar,
    state varchar,
    country varchar,
    postalCode varchar,
    territory varchar
)

CREATE TABLE orderdetails (
    orderNumber int,
    productCode varchar,
    quantityOrdered int,
    priceEach decimal,
    orderLineNumber smallint
)

CREATE TABLE orders (
    orderNumber int,
    orderDate date,
    requi

In [6]:
username = 'postgres'   
password = ''  
hostname = 'localhost'
database_name = 'companydb'  

sql_agent = DatabaseAgent(host_name=hostname,port=5432,username=username,database_name=database_name,password=password, database_type="PostgreSQL")

Connection to MySQL database successful!


In [7]:
sql_agent.grab_table_schema(sql_agent.grab_table_names())

[Table(name='departments', columns=[TableColumn(name='id', dtype='integer'), TableColumn(name='name', dtype='character varying'), TableColumn(name='location', dtype='character varying')], pks=None, fks=None),
 Table(name='employees', columns=[TableColumn(name='id', dtype='integer'), TableColumn(name='salary', dtype='integer'), TableColumn(name='department_id', dtype='integer'), TableColumn(name='name', dtype='character varying'), TableColumn(name='position', dtype='character varying')], pks=None, fks=None)]